In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
# ------------------------------------
# 1) Define your portfolio tickers
# ------------------------------------

# Define your portfolio tickers
portfolio_ticker = [
    "AAPL", "JPM", "NFLX", "CSCO", "GE", "DHR", "ANET", "UNP", "FI", "VRTX",
    "NVDA", "LLY", "BAC", "MS", "AMD", "INTU", "C", "UBER", "MU", "MRVL",
    "MSFT", "V", "JNJ", "NOW", "ABT", "PLTR", "PFE", "CMCSA", "LMT", "SBUX",
    "GOOG", "XOM", "CRM", "AXP", "GS", "VZ", "LOW", "KKR", "PANW", "MMC",
    "AMZN", "MA", "ABBV", "BX", "DIS", "BKNG", "AMGN", "SCHW", "APP", "NKE",
    "META", "UNH", "CVX", "TMO", "PM", "RTX", "SYK", "TJX", "GILD", "PLD",
    "TSLA", "ORCL", "KO", "ISRG", "ADBE", "T", "NEE", "COP", "BMY", "LRCX",
    "AVGO", "COST", "WFC", "IBM", "CAT", "AMAT", "BSX", "BA", "UPS", "KLAC",
    "BRK-B", "HD", "TMUS", "PEP", "QCOM", "SPGI", "HON", "DE", "GEV", "CHTR",
    "WMT", "PG", "MRK", "MCD", "TXN", "BLK", "PGR", "ADP", "ADI", "CEG"
]


In [3]:
# ------------------------------------
# 2) Signal generation function
# ------------------------------------
def signal_generator(df):
    """
    Given two consecutive rows (df.iloc[0] and df.iloc[1]),
    return:
        1 for a Bearish pattern  (Sell),
        2 for a Bullish pattern (Buy),
        0 otherwise.
    """
    # Extract scalar values (ensure they're floats)
    open_price     = float(df.iloc[1]["Open"])
    close_price    = float(df.iloc[1]["Close"])
    previous_open  = float(df.iloc[0]["Open"])
    previous_close = float(df.iloc[0]["Close"])

    # Bearish Pattern
    if (open_price > close_price
        and previous_open < previous_close
        and close_price < previous_open
        and open_price >= previous_close):
        return 1  # Bearish (Sell signal)

    # Bullish Pattern
    elif (open_price < close_price
          and previous_open > previous_close
          and close_price > previous_open
          and open_price <= previous_close):
        return 2  # Bullish (Buy signal)

    # No clear pattern
    else:
        return 0

In [4]:
# ------------------------------------
# 3) Fetch data from yfinance
# ------------------------------------
def fetch_data(ticker, interval, period):
    """
    Fetch historical stock data for a given ticker using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(ticker)
        print(f"📥 Fetching historical data for {ticker}...")
        data = stock.history(interval=interval, period=period)  

        if data.empty:
            print(f"⚠️ No data found for {ticker}.")
            return None
        return data

    except Exception as e:
        print(f"❌ Could not fetch data for {ticker}: {e}")
        return None


In [5]:
# ------------------------------------
# 4) Generate and recive signals from the data
# ------------------------------------
def evaluate_algorithm(interval='30m', period="1mo"):
    """
    Evaluate trading signals for all tickers in the portfolio.
    """
    failed_tickers = []
    evaluation_results = {}
    price_data = {}

    for ticker in portfolio_ticker:
        dataF = fetch_data(ticker, interval, period)

        # ✅ Check for missing data
        if dataF is None or dataF.empty or len(dataF) < 2:
            print(f"⚠️ Not enough data for {ticker}. Skipping.")
            failed_tickers.append(ticker)
            continue

        # ✅ Generate signals
        signals = [0]  # First row can't have a signal
        for i in range(1, len(dataF)):
            df_slice = dataF.iloc[i - 1: i + 1]
            signals.append(signal_generator(df_slice))

        dataF["Signal"] = signals

        # ✅ Save data with signals
        price_data[ticker] = dataF[["Close", "Signal"]]
        evaluation_results[ticker] = {
            "Total Signals": dataF["Signal"].value_counts().to_dict(),
            "Latest Signal": dataF["Signal"].iloc[-1]
        }

        print(f"✅ Processed {ticker}: {evaluation_results[ticker]}")

    # ✅ Summary of failed tickers
    if failed_tickers:
        print(f"\n⚠️ Failed to process: {failed_tickers}")

    return price_data  # Dictionary with ticker-wise time series




In [6]:
# ------------------------------------
# 5) Prompt for portfolio size
# ------------------------------------
def get_portfolio_size():
    """
    Prompt user for a portfolio size and convert it to float.
    Retries if user does not provide a valid number.
    """
    while True:
        try:
            val = float(input("Enter the value of your portfolio: "))
            return val
        except ValueError:
            print("That's not a number! Try again.\n")

In [7]:
# ------------------------------------
# 6) Dynamic portfolio alocation following
# the long or short signals for every stock 
# ------------------------------------

def track_portfolio_over_time(price_data, initial_capital):
    """
    Simulates portfolio performance over time.
    - Tracks daily portfolio value and composition separately.
    - Adjusts holdings based on buy/sell signals.
    - Logs changes in cash balance, asset allocation, and asset value.
    """
    portfolio_value_data = []
    portfolio_composition_data = []
    asset_value_data = []  # New table to track total value of each asset
    
    cash_balance = float(initial_capital)  # Ensure cash is a float
    holdings = {ticker: 0 for ticker in price_data.keys()}  # Start with 0 shares

    # Get all unique dates from all assets
    all_dates = sorted(set(date for data in price_data.values() for date in data.index))

    for date in all_dates:
        date = pd.Timestamp(date)  # Ensure the date is a pandas Timestamp
        total_value = cash_balance  # Start with available cash

        # Track daily holdings and asset values
        daily_holdings = {ticker: 0 for ticker in price_data.keys()}
        daily_asset_values = {ticker: 0 for ticker in price_data.keys()}

        for ticker, data in price_data.items():
            if date not in data.index:
                continue  # Skip missing data

            close_price = float(data.loc[date, "Close"])  # Ensure a float value
            
            # Ensure "Signal" is a single scalar value
            signal_value = data.loc[date, "Signal"]
            if isinstance(signal_value, pd.Series):  
                signal_value = signal_value.iloc[0]
            signal = int(signal_value)  # Convert to integer

            # ✅ Dynamic Allocation Based on Active Signals
            active_buy_signals = sum(1 for t in price_data if price_data[t].loc[date, "Signal"] == 2)
            position_size = 0.25  # Allocate 25% of cash per position

            if signal == 2 and cash_balance > 0:
                # Buy only if there are active buy signals
                if active_buy_signals > 0:
                    shares_to_buy = (cash_balance * position_size) / close_price
                    holdings[ticker] += shares_to_buy
                    cash_balance -= shares_to_buy * close_price

            elif signal == 1:
                # Sell fully if a sell signal is triggered
                cash_balance += holdings[ticker] * close_price
                holdings[ticker] = 0

            # Compute total portfolio value
            total_value += holdings[ticker] * close_price
            daily_holdings[ticker] = holdings[ticker]  # Track number of shares
            daily_asset_values[ticker] = holdings[ticker] * close_price  # Track asset value

        # Compute percentage allocation
        allocation = {
            ticker: (holdings[ticker] * price_data[ticker].loc[date, "Close"]) / total_value 
            if total_value > 0 else 0
            for ticker in holdings
        }

        # Store daily portfolio value
        portfolio_value_data.append({
            "Date": date,
            "Portfolio Value": total_value,
            "Cash Balance": cash_balance
        })

        # Store daily portfolio composition
        portfolio_composition_data.append({
            "Date": date,
            **daily_holdings,  # Number of shares per asset
            **{f"Alloc_{ticker}": round(alloc, 4) for ticker, alloc in allocation.items()}  # Clean % Allocation
        })

        # Store daily asset value
        asset_value_data.append({
            "Date": date,
            **daily_asset_values  # Total value of each asset
        })

    # Convert lists to DataFrames
    portfolio_value_df = pd.DataFrame(portfolio_value_data).set_index("Date")
    portfolio_composition_df = pd.DataFrame(portfolio_composition_data).set_index("Date")
    asset_value_df = pd.DataFrame(asset_value_data).set_index("Date")  # New table

    return portfolio_value_df, portfolio_composition_df, asset_value_df  # Return three tables




In [8]:
# ------------------------------------
# 7) Run everything and build the portfolio value frame
# ------------------------------------

def main():
    # Fetch signals and price data
    price_data = evaluate_algorithm()

    # Ask user for initial capital
    initial_capital = get_portfolio_size()

    # Track portfolio performance over time
    portfolio_value_df, portfolio_composition_df, asset_value_df = track_portfolio_over_time(price_data, initial_capital)

    # Calculate returns
    portfolio_value_df["Daily Return"] = portfolio_value_df["Portfolio Value"].pct_change()
    portfolio_value_df["Cumulative Return"] = (1 + portfolio_value_df["Daily Return"]).cumprod()

    print("\n📊 Portfolio Value Evolution:")
    print(portfolio_value_df.tail())  # Show last few days of total portfolio value

    print("\n📈 Portfolio Composition Over Time:")
    print(portfolio_composition_df.tail())  # Show last few days of portfolio allocation

    print("\n💰 Asset Value Over Time:")
    print(asset_value_df.tail())  # Show last few days of asset values

    return portfolio_value_df, portfolio_composition_df, asset_value_df  # Return all three tables




In [9]:
portfolio_value_df, portfolio_composition_df, asset_value_df = main()

📥 Fetching historical data for AAPL...
✅ Processed AAPL: {'Total Signals': {0: 255, 2: 18, 1: 13}, 'Latest Signal': 2}
📥 Fetching historical data for JPM...
✅ Processed JPM: {'Total Signals': {0: 251, 2: 18, 1: 17}, 'Latest Signal': 0}
📥 Fetching historical data for NFLX...
✅ Processed NFLX: {'Total Signals': {0: 253, 2: 18, 1: 15}, 'Latest Signal': 0}
📥 Fetching historical data for CSCO...
✅ Processed CSCO: {'Total Signals': {0: 240, 1: 23, 2: 23}, 'Latest Signal': 2}
📥 Fetching historical data for GE...
✅ Processed GE: {'Total Signals': {0: 253, 1: 18, 2: 15}, 'Latest Signal': 0}
📥 Fetching historical data for DHR...
✅ Processed DHR: {'Total Signals': {0: 242, 1: 24, 2: 20}, 'Latest Signal': 0}
📥 Fetching historical data for ANET...
✅ Processed ANET: {'Total Signals': {0: 254, 2: 19, 1: 13}, 'Latest Signal': 0}
📥 Fetching historical data for UNP...
✅ Processed UNP: {'Total Signals': {0: 250, 1: 21, 2: 15}, 'Latest Signal': 0}
📥 Fetching historical data for FI...
✅ Processed FI: {'Tot

In [10]:
portfolio_value_df

,Portfolio Value,Cash Balance,Daily Return,Cumulative Return
Date,,,,
2025-01-15 09:30:00-05:00,100000.000000,100000.000000,NaN,NaN
2025-01-15 10:00:00-05:00,176269.531250,23730.468750,0.762695,1.762695
2025-01-15 10:30:00-05:00,85952.029995,37599.578178,-0.512383,0.859520
2025-01-15 11:00:00-05:00,128500.119450,8922.556150,0.495021,1.285001
2025-01-15 11:30:00-05:00,107938.905718,669.947329,-0.160009,1.079389
...,...,...,...,...
2025-02-14 13:30:00-05:00,106747.925516,3001.828990,0.004583,1.067479
2025-02-14 14:00:00-05:00,101319.468642,9062.829924,-0.050853,1.013195
2025-02-14 14:30:00-05:00,114544.787252,1903.095829,0.130531,1.145448


In [11]:
portfolio_composition_df

,AAPL,JPM,NFLX,CSCO,GE,DHR,ANET,UNP,FI,VRTX,...,Alloc_WMT,Alloc_PG,Alloc_MRK,Alloc_MCD,Alloc_TXN,Alloc_BLK,Alloc_PGR,Alloc_ADP,Alloc_ADI,Alloc_CEG
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-15 09:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000
2025-01-15 10:00:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000
2025-01-15 10:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.1458
2025-01-15 11:00:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0977
2025-01-15 11:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.1166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14 13:30:00-05:00,2.072740,5.308971,2.131820,109.977425,0.000000,0.0,1.938285,11.628994,0.0,6.275309,...,0.0,0.0199,0.0000,0.0,0.0213,0.0,0.0,0.0,0.0072,0.0153
2025-02-14 14:00:00-05:00,2.072740,5.308971,2.131820,109.977425,0.000000,0.0,1.938285,0.000000,0.0,0.000000,...,0.0,0.0000,0.0000,0.0,0.0223,0.0,0.0,0.0,0.0076,0.0161
2025-02-14 14:30:00-05:00,11.348228,5.308971,2.131820,109.977425,8.155112,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0000,0.0055,0.0,0.0197,0.0,0.0,0.0,0.0067,0.0142


In [12]:
asset_value_df

,AAPL,JPM,NFLX,CSCO,GE,DHR,ANET,UNP,FI,VRTX,...,WMT,PG,MRK,MCD,TXN,BLK,PGR,ADP,ADI,CEG
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-15 09:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2025-01-15 10:00:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
2025-01-15 10:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,12533.192726
2025-01-15 11:00:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,12552.262142
2025-01-15 11:30:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,12587.971911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14 13:30:00-05:00,506.080263,1476.106466,2252.395695,7151.282232,0.000000,0.0,207.008796,2918.877456,0.0,2891.222994,...,0.0,2119.755222,0.000000,0.0,2270.823598,0.0,0.0,0.0,765.119902,1628.368864
2025-02-14 14:00:00-05:00,506.059515,1475.655086,2256.316869,7143.583846,0.000000,0.0,207.318914,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,2264.037436,0.0,0.0,0.0,765.030298,1626.357947
2025-02-14 14:30:00-05:00,2772.012204,1475.257011,2256.019684,7133.312059,1699.280611,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,634.365276,0.0,2260.705920,0.0,0.0,0.0,763.685907,1629.709317
